In [83]:
# example of pix2pix gan for satellite to map image-to-image translation
# example of pix2pix gan for satellite to map image-to-image translation
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
#from tensorflow import keras 
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model,  load_model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv1D, UpSampling1D
# from keras.layers import Conv2D
# from keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU,ZeroPadding1D
from tensorflow.keras.layers import ReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LeakyReLU
from matplotlib import pyplot as plt
import numpy as np
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import Add

#--- 20210820:存檔錄加上日期 ---#
import datetime
from time import time
import os
import sys


# define the discriminator model# 定義判別器模型
def define_discriminator(image_shape):
    # weight initialization權重初始化
    init = RandomNormal(stddev=0.02)
    # source image input 源圖像輸入
    in_src_image = Input(shape=image_shape)
    # target image input 目標圖像輸入
    in_target_image = Input(shape=image_shape)
    # concatenate images channel-wise 按通道連接圖像
    merged = Concatenate()([in_src_image, in_target_image]) #merged 合併 
    # C64
    d = Conv1D(64, 4, strides=2, padding='same', kernel_initializer=init)(merged)
    # d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    # C128
    d = Conv1D(128, 4, strides=2, padding='same', kernel_initializer=init)(d)
    # d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C256
    d = Conv1D(256, 4, strides=2, padding='same', kernel_initializer=init)(d)
    # d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C512
    d = Conv1D(512, 4, strides=2, padding='same', kernel_initializer=init)(d)
    # d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # second last output layer
    d = Conv1D(512, 4, padding='same', kernel_initializer=init)(d)
    # d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # patch output
    d = Conv1D(1, 4, padding='same', kernel_initializer=init)(d)
    # d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    patch_out = Activation('sigmoid')(d)
    # define model
    model = Model([in_src_image, in_target_image], patch_out)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    return model
model = define_discriminator((10,10,3))

/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


In [84]:
import numpy as np

def generate_input_data(image_shape):
    # 生成隨機的源圖像和目標圖像
    src_image = np.random.random(image_shape)
    target_image = np.random.random(image_shape)
    return [src_image, target_image]

# 生成一個輸入資料
input_data = np.array(generate_input_data((1,10, 10, 3)))
target_image = np.array(generate_input_data((1,10, 10, 3)))
predictions = model.predict([input_data[0], target_image[0]])
predictions.shape

1/1 [==============================] - 0s 283ms/step


(1, 10, 1, 1)

In [85]:
import torch
import torch.nn as nn
class Discriminator(nn.Module):
    def __init__(self, image_shape):
        super(Discriminator, self).__init__()

        in_channels = image_shape[0]
        self.model = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=4, stride=2, padding=1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(512, 1, kernel_size=4, stride=1, padding=1),
            nn.Sigmoid()
        )

    def forward(self, in_src_image, in_target_image):
        merged = torch.cat((in_src_image, in_target_image), dim=-1)
        print(merged.shape)
        output = self.model(merged)
        return output
model = Discriminator((1, 1, 100, 100))
import numpy as np
def generate_input_data(image_shape):
    # 生成隨機的源圖像和目標圖像
    src_image = np.random.random(image_shape)
    target_image = np.random.random(image_shape)
    return [src_image, target_image]
input_data = generate_input_data((1, 1, 100, 100))  # Set the number of channels to 1
target_image = generate_input_data((1, 1, 100, 100))  # Set the number of channels to 1
predictions = model(torch.FloatTensor(input_data[0]), torch.FloatTensor(target_image[0]))
predictions

torch.Size([1, 1, 100, 200])


tensor([[[[0.5755, 0.5750, 0.6497, 0.6327, 0.5123, 0.6442, 0.5410, 0.6694,
           0.5607, 0.6365, 0.6483],
          [0.5547, 0.6836, 0.4459, 0.4319, 0.5463, 0.5634, 0.5797, 0.5581,
           0.3968, 0.6621, 0.3627],
          [0.5635, 0.4801, 0.5313, 0.5427, 0.5513, 0.5401, 0.7156, 0.5752,
           0.6434, 0.6097, 0.5944],
          [0.4976, 0.6121, 0.5637, 0.6088, 0.6037, 0.4627, 0.6364, 0.4505,
           0.6125, 0.3340, 0.5124],
          [0.5690, 0.7199, 0.4382, 0.6627, 0.5186, 0.5586, 0.6566, 0.5177,
           0.4470, 0.4945, 0.6430]]]], grad_fn=<SigmoidBackward0>)